In [1]:
#!pip install ray
#!pip install modin[ray] 

**Import libs**

In [2]:
import pandas as pd
import ray
import operator
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from google.colab import files


**Loading Dataset and Partitioning**

In [3]:
uploaded = files.upload()

Saving data.data to data (1).data


In [4]:
l = int(input("# of Dataset records = "))
n = int(input("Number of Partitions = "))
s = int(l/n)
#=================================================

#=================================================
dataset = []
for i,chunk in enumerate(pd.read_csv('/content/data.data' , chunksize=s)):
  dataset.append(chunk.astype('category'))
#print(dataset[2].info())
X,Y = [],[]
for i in range(s):
  X.append(dataset[0].iloc[:,0:6])
  Y.append(dataset[0].iloc[:,6:76])

# of Dataset records = 1728
Number of Partitions = 5


In [5]:
#kf = KFold(n_splits=10, random_state=1, shuffle=True)
#X_train, X_test, Y_train, Y_test = [],[],[],[]
#for i in range(s):
#  for train_index, test_index in kf.split(X[i]):
#     X_train, X_test = X[i][train_index], X[i][test_index]
#     y_train, y_test = Y[i][train_index], Y[i][test_index]

In [6]:
#====================================================
ray.init()

RayContext(dashboard_url='', python_version='3.7.13', ray_version='1.13.0', ray_commit='e4ce38d001dbbe09cd21c497fedd03d692b2be3e', address_info={'node_ip_address': '172.28.0.2', 'raylet_ip_address': '172.28.0.2', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-06-27_15-00-30_848012_1416/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-06-27_15-00-30_848012_1416/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-06-27_15-00-30_848012_1416', 'metrics_export_port': 34650, 'gcs_address': '172.28.0.2:59629', 'address': '172.28.0.2:59629', 'node_id': 'c6d6c951c3d9b7eed52c1edf6da0c007e1896b9b8fbf3591787b0d0e'})

**Define Ray classes and remotes**

In [7]:
@ray.remote
class Racer(object):

  rules = {}
  final_rules = {}
  columns = {}
  classes = []

  # Fitness Value = alpha * accuracy + beta * coverage
  # accuracy coef : accuracy is the percent of covered instances which are correctly classified 
  # coverage coef : coverage is the percent of instances which are covered among the training set


  rules_size = 0
  d_size = 0
  X = None
  Y = None
  def __init__(self, alpha, beta,X,Y):  # initialize valiables
    self.alpha = alpha
    self.beta = beta
    self.rules = {}
    self.final_rules = {}
    self.columns = {}
    self.classes = []
    self.X = X
    self.Y = Y
    self.rules_size = 0
    self.d_size = 0
  
  def fit(self):
    X = self.X
    Y = self.Y
    keys = Y.keys()

    if Y[keys[0]].dtype.name != "category":
      raise ValueError(f'Y must be of type "category", "{Y.loc[0].dtype.name}" given')
    for i in Y[keys[0]].cat.categories:
      self.classes.append(i)
      self.rules[i] = []
    
    cols = X.columns.copy()
    self.d_size = X.shape[0]
    for c in cols:
      if X[c].dtype.name != "category":
        raise ValueError(f'All columns must be of type "category", "{X[c].dtype.name}" given')
      self.columns[c] = []
      for i in X[c].cat.categories:
        self.columns[c].append(i)
    
    self.X = X.copy()
    self.Y = Y.copy()
    # self.rules_size += len(self.classes)
    for key in self.columns:
      self.rules_size += len(self.columns[key])

    self.convert_rules()

    self.proccess_rules()


  
  def proccess_rules(self):
    for c in self.classes:
      extant_rules = self.rules[c].copy()
      extant_rules = [[i,0] for i in extant_rules]
      for i in range(len(extant_rules)):
        for j in range(i+1, len(extant_rules)):
          if extant_rules[i][1] == 0 and extant_rules[j][1] == 0:
            composed_rule = self.composition(extant_rules[i][0], extant_rules[j][0])
            if self.fitness(composed_rule, c) > self.fitness(extant_rules[i][0], c) and self.fitness(composed_rule, c) > self.fitness(extant_rules[j][0], c):
              extant_rules[i][0] = composed_rule
              extant_rules[j][1] = 1
              for x in range(len(extant_rules)):
                if x != i and self.rule_covers(composed_rule, extant_rules[x][0]):
                  extant_rules[x][1] = 1
      
      for i in range(len(extant_rules)):
        if extant_rules[i][1] == 0:
          extant_rules[i][0] = self.generalize(extant_rules[i][0], c)
      
      final = sorted(extant_rules, key=lambda x: self.fitness(x[0], c), reverse=True)
      self.final_rules[c] = [[i[0], self.fitness(i[0], c)] for i in final if i[1] == 0]
      # self.final_rules[c] = final.copy()

  
  def predict(self, X):
    cols = X.columns.copy()
    Y = []
    for c in cols:
      if X[c].dtype.name != "category":
        raise ValueError(f'All columns must be of type "category", "{X[c].dtype.name}" given')
    
    for i in range(X.shape[0]):
      rule = self.generate_rule(X.iloc[i])
      rules = []
      for c in self.classes:
        for j in range(len(self.final_rules[c])):
          if self.rule_covers(self.final_rules[c][j][0], rule):
            rules.append([c, self.final_rules[c][j][1], j])
      rules.sort(key=operator.itemgetter(1, 2), reverse=True)
      if len(rules)==0:
        Y.append(-1)
        continue
      Y.append(rules[0][0])
    
    return Y

    
  def generalize(self, rule, c):
    for i in range(len(rule)):
      if rule[i] == 0:
        new_rule = rule
        new_rule[i] = 1
        if self.fitness(new_rule, c) > self.fitness(rule, c):
          rule = new_rule
    return rule

  def composition(self, rule1, rule2):
    result = ""
    for i in range(self.rules_size):
      if rule1[i] == "1" or rule2[i] == "1":
        result += "1"
      else:
        result += "0"
    
    return result
  
  def fitness(self, rule, cls):
    return self.alpha * self.accuracy(rule, cls) + self.beta * self.coverage(rule)
  
  def accuracy(self, rule, cls):
    return self.n_correct(rule, cls) / self.n_covers(rule)
  
  def coverage(self, rule):
    return self.n_covers(rule) / self.d_size

  def n_correct(self, rule1, cls):
    num = 0
    for rule in self.rules[cls]:
      correct = True
      for i in range(self.rules_size):
        if rule1[i] != rule[i] and rule1[i] == "0":
          correct = False
      if correct:
        num += 1
    
    return num

  def n_covers(self, rule1):
    num = 0
    for cls in self.classes:
      for rule in self.rules[cls]:
        correct = True
        for i in range(self.rules_size):
          if rule1[i] != rule[i] and rule1[i] == "0":
            correct = False
        if correct:
          num += 1
    
    return num


  def rule_covers(self, rule1, rule2):
    for i in range(self.rules_size):
      if rule2[i] == "1" and rule1[i] == "0":
        return False
    return True


  def convert_rules(self):
    for i in range(self.X.shape[0]):
      self.generate_rules(self.X.iloc[i], self.Y.iloc[i])

  def generate_rule(self, input):
    rule = ""
    for key in self.columns:
      sub_rule = ""
      for i in self.columns[key]:
        if i == input[key]:
          sub_rule += "1"
        else:
          sub_rule += "0"
      rule += sub_rule
    return rule
  
  def generate_rules(self, input, output):
    rule = ""
    for key in self.columns:
      sub_rule = ""
      for i in self.columns[key]:
        if i == input[key]:
          sub_rule += "1"
        else:
          sub_rule += "0"
      rule += sub_rule
    rule_class = None
    for i in self.classes:
      if (i == output).bool():
        rule_class = i

    
    #self.rules[rule_class].append(rule)
    self.rules[rule_class].append(rule+'->'+str(rule_class))

  def get_final_rules(self):
    return self.final_rules



In [8]:
#====================================================
machines = [Racer.remote(alpha=0.7,beta=0.3,X=X[i],Y=Y[i]) for i in range(5)]
machines

[Actor(Racer, c313ebce9b9967ce90470b8501000000),
 Actor(Racer, c5a387fb9630c46bf706c31701000000),
 Actor(Racer, e2706e07bf136d91482421ce01000000),
 Actor(Racer, fb6aa99f2a5ae77a5a4c49ac01000000),
 Actor(Racer, 41e2362bab5b2d7dc9f7ed6e01000000)]

**Call fit function**

In [9]:
ray.get([m.fit.remote() for i,m in enumerate(machines)])

[None, None, None, None, None]

In [10]:
rules = ray.get([m.get_final_rules.remote() for m in machines])
rules

[{'acc': [['101101111011100101', 0.7156521739130435],
   ['101101111010111100', 0.7130434782608696],
   ['100101111001010100', 0.7034782608695652],
   ['100100111001001100', 0.7026086956521739],
   ['100100111001010001', 0.7026086956521739],
   ['100100011010010001', 0.7017391304347825]],
  'unacc': [['111111111111111111', 0.9188405797101449]]},
 {'acc': [['101101111011100101', 0.7156521739130435],
   ['101101111010111100', 0.7130434782608696],
   ['100101111001010100', 0.7034782608695652],
   ['100100111001001100', 0.7026086956521739],
   ['100100111001010001', 0.7026086956521739],
   ['100100011010010001', 0.7017391304347825]],
  'unacc': [['111111111111111111', 0.9188405797101449]]},
 {'acc': [['101101111011100101', 0.7156521739130435],
   ['101101111010111100', 0.7130434782608696],
   ['100101111001010100', 0.7034782608695652],
   ['100100111001001100', 0.7026086956521739],
   ['100100111001010001', 0.7026086956521739],
   ['100100011010010001', 0.7017391304347825]],
  'unacc': [['